### Libraries:

In [ ]:
# !pip install sklearn-genetic-opt

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
import time

from sklearn import preprocessing

from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials

from sklearn_genetic import GASearchCV
from sklearn.model_selection import KFold
from sklearn_genetic.space import Categorical, Integer, Continuous

### Warnings:

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

### Read dataset:

In [3]:
df = pd.read_csv('data1_filled.csv')
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347,0.124451,0.166405
1,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335,0.124451,0.166405
2,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500,0.124451,0.166405
3,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129,0.124451,0.166405
4,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347,0.124451,0.166405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107845,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,200,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527,0.124735,0.721234
107846,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,201,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992,0.223992,0.539739
107847,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,202,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993,0.124735,0.733096
107848,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,203,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297,0.124204,0.717675


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import pandas as pd
# import io

# df = pd.read_csv(io.BytesIO(uploaded['data1_filled.csv']))
# df

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107850 entries, 0 to 107849
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Study_Folder    107850 non-null  object 
 1   Subject_Folder  107850 non-null  object 
 2   Stimuli         107850 non-null  int64  
 3   Frame Number    107850 non-null  int64  
 4   Face_Y          107850 non-null  float64
 5   Face_X          107850 non-null  float64
 6   Face_W          107850 non-null  float64
 7   Face_H          107850 non-null  float64
 8   Right_Eye_X     107850 non-null  float64
 9   Right_Eye_Y     107850 non-null  float64
 10  Left_Eye_X      107850 non-null  float64
 11  Left_Eye_Y      107850 non-null  float64
 12  Face_Angle      107850 non-null  float64
 13  X_Predictions   107850 non-null  float64
 14  Y_Predictions   107850 non-null  float64
dtypes: float64(11), int64(2), object(2)
memory usage: 12.3+ MB


### Drop columns:

In [5]:
df.drop(['Study_Folder','Subject_Folder'], axis = 1, inplace = True)

In [6]:
df

,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0,0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347,0.124451,0.166405
1,0,1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335,0.124451,0.166405
2,0,2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500,0.124451,0.166405
3,0,3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129,0.124451,0.166405
4,0,4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347,0.124451,0.166405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107845,9806,200,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527,0.124735,0.721234
107846,9806,201,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992,0.223992,0.539739
107847,9806,202,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993,0.124735,0.733096
107848,9806,203,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297,0.124204,0.717675


In [7]:
# Checking NaN values
df.isna().sum()

Stimuli          0
Frame Number     0
Face_Y           0
Face_X           0
Face_W           0
Face_H           0
Right_Eye_X      0
Right_Eye_Y      0
Left_Eye_X       0
Left_Eye_Y       0
Face_Angle       0
X_Predictions    0
Y_Predictions    0
dtype: int64

### Define Stimuli, X,Y:

In [8]:
df.columns

Index(['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W', 'Face_H',
       'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle',
       'X_Predictions', 'Y_Predictions'],
      dtype='object')

In [9]:
features_columns = ['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W','Face_H',
                    'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle']

target_columns=['X_Predictions', 'Y_Predictions']

In [10]:
X=df[features_columns]

In [11]:
y=df[target_columns]

In [12]:
X_train_valid= X[X['Stimuli']<= 11]

In [13]:
X_test= X[X['Stimuli']> 11]

In [14]:
y_train_valid= y[X['Stimuli']<= 11]

In [15]:
y_test= y[X['Stimuli']> 11]

### Split data:

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.3,stratify =X_train_valid['Stimuli'] ,random_state=42)

In [17]:
print("Whole dataset :",len(df))
print("X Train size", len(X_train))
print("y Train size", len(y_train))
print("X Valid size", len(X_valid))
print("y Valid size", len(y_valid))
print("X Test size", len(X_test))
print("y Test size", len(y_test))

Whole dataset : 107850
X Train size 38986
y Train size 38986
X Valid size 16709
y Valid size 16709
X Test size 52155
y Test size 52155


In [18]:
print("X Train shape", X_train.shape)
print("y Train shape", y_train.shape)
print("X Valid shape", X_valid.shape)
print("y Valid shape", y_valid.shape)
print("X Test shape", X_test.shape)
print("y Test shape", y_test.shape)

X Train shape (38986, 11)
y Train shape (38986, 2)
X Valid shape (16709, 11)
y Valid shape (16709, 2)
X Test shape (52155, 11)
y Test shape (52155, 2)


In [19]:
X_train['Stimuli'].value_counts().sort_values()/len(X_train), X_valid['Stimuli'].value_counts().sort_values()/len(X_valid)

(0     0.081542
 11    0.082389
 5     0.082389
 6     0.083158
 2     0.083671
 7     0.083748
 3     0.083748
 10    0.083774
 8     0.083774
 4     0.083799
 9     0.083825
 1     0.084184
 Name: Stimuli, dtype: float64,
 0     0.081513
 5     0.082351
 11    0.082411
 6     0.083189
 2     0.083667
 10    0.083727
 7     0.083727
 8     0.083787
 3     0.083787
 4     0.083787
 9     0.083847
 1     0.084206
 Name: Stimuli, dtype: float64)

### Drop columns:

In [20]:
drop_column = ['Stimuli','Frame Number']

In [21]:
X.drop(columns = drop_column, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_15848\3791142718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns = drop_column, inplace=True)


In [22]:
X

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347
1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335
2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500
3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129
4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347
...,...,...,...,...,...,...,...,...,...
107845,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527
107846,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992
107847,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993
107848,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297


In [23]:
X_train.drop(columns = drop_column, inplace=True)

In [24]:
X_train

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
8500,194.0,182.0,237.0,237.0,274.0,361.0,285.0,273.0,0.303291
34560,241.0,214.0,196.0,196.0,290.0,378.0,295.0,307.0,0.572514
49079,233.0,89.0,251.0,251.0,191.0,408.0,195.0,317.0,2.094841
53301,255.0,178.0,262.0,262.0,284.0,429.0,284.0,342.0,0.450649
31581,200.0,127.0,217.0,217.0,205.0,353.0,208.0,268.0,0.608209
...,...,...,...,...,...,...,...,...,...
42620,145.0,314.0,189.0,189.0,386.0,272.0,383.0,211.0,0.176726
24503,264.0,230.0,180.0,180.0,300.0,386.0,303.0,329.0,0.535068
34795,178.0,207.0,215.0,215.0,291.0,310.0,291.0,243.0,0.267951
50664,284.0,211.0,205.0,205.0,293.0,426.0,295.0,355.0,0.584136


In [25]:
X_valid.drop(columns = drop_column, inplace=True)

In [26]:
X_valid

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
1879,203.0,76.0,222.0,222.0,167.0,369.0,163.0,283.0,0.731607
54927,195.0,189.0,230.0,230.0,282.0,349.0,284.0,263.0,0.291002
26246,239.0,248.0,214.0,214.0,336.0,391.0,332.0,304.0,0.278280
51101,191.0,158.0,232.0,232.0,243.0,347.0,255.0,262.0,0.381798
54715,204.0,158.0,248.0,248.0,260.0,374.0,259.0,267.0,0.309640
...,...,...,...,...,...,...,...,...,...
32028,283.0,205.0,203.0,203.0,286.0,405.0,289.0,351.0,0.602547
23295,236.0,194.0,204.0,204.0,271.0,379.0,272.0,302.0,0.477154
43401,308.0,77.0,247.0,247.0,173.0,470.0,171.0,393.0,1.450301
49843,277.0,200.0,243.0,243.0,295.0,431.0,305.0,346.0,0.510805


In [27]:
X_test.drop(columns = drop_column, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_15848\2944433928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns = drop_column, inplace=True)


In [28]:
X_test

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
55695,241.0,220.0,194.0,194.0,288.0,379.0,288.0,297.0,0.424277
55696,242.0,221.0,193.0,193.0,288.0,379.0,288.0,297.0,0.421167
55697,242.0,221.0,193.0,193.0,287.0,379.0,287.0,297.0,0.421158
55698,241.0,221.0,193.0,193.0,287.0,379.0,287.0,297.0,0.421158
55699,240.0,221.0,195.0,195.0,288.0,379.0,287.0,296.0,0.421158
...,...,...,...,...,...,...,...,...,...
107845,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527
107846,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992
107847,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993
107848,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297


### MinMaxScaler:

In [29]:
minmax_scaler = preprocessing.MinMaxScaler()
X_train_scal = minmax_scaler.fit_transform(X_train)
X_valid_scal = minmax_scaler.transform(X_valid)
X_test_scal = minmax_scaler.transform(X_test)

In [30]:
X_train_scal

array([[0.3874092 , 0.53246753, 0.39884393, ..., 0.57638889, 0.39793282,
        0.00243009],
       [0.50121065, 0.63636364, 0.16184971, ..., 0.61111111, 0.48578811,
        0.00624757],
       [0.48184019, 0.23051948, 0.47976879, ..., 0.26388889, 0.51162791,
        0.02783358],
       ...,
       [0.34866828, 0.61363636, 0.2716763 , ..., 0.59722222, 0.32041344,
        0.00192898],
       [0.60532688, 0.62662338, 0.21387283, ..., 0.61111111, 0.60981912,
        0.00641237],
       [0.66101695, 0.18831169, 0.39306358, ..., 0.16319444, 0.67183463,
        0.01818552]])

In [31]:
X_valid_scal

array([[0.40920097, 0.18831169, 0.31213873, ..., 0.15277778, 0.42377261,
        0.00850345],
       [0.38983051, 0.55519481, 0.3583815 , ..., 0.57291667, 0.37209302,
        0.00225584],
       [0.49636804, 0.74675325, 0.26589595, ..., 0.73958333, 0.47803618,
        0.00207545],
       ...,
       [0.66343826, 0.19155844, 0.4566474 , ..., 0.18055556, 0.70801034,
        0.01869425],
       [0.58837772, 0.59090909, 0.43352601, ..., 0.64583333, 0.58656331,
        0.00537255],
       [0.3559322 , 0.23701299, 0.65895954, ..., 0.33680556, 0.37209302,
        0.00323463]])

In [32]:
X_test_scal

array([[0.50121065, 0.65584416, 0.15028902, ..., 0.58680556, 0.45994832,
        0.00414562],
       [0.50363196, 0.65909091, 0.14450867, ..., 0.58680556, 0.45994832,
        0.00410153],
       [0.50363196, 0.65909091, 0.14450867, ..., 0.58333333, 0.45994832,
        0.00410139],
       ...,
       [0.40193705, 0.46753247, 0.3699422 , ..., 0.47569444, 0.39018088,
        0.00361695],
       [0.40193705, 0.45779221, 0.39306358, ..., 0.47222222, 0.39018088,
        0.00308244],
       [0.40677966, 0.47077922, 0.3583815 , ..., 0.47569444, 0.39018088,
        0.00347246]])

### Metrics:

In [33]:
metric_dict = {"MAE":mean_absolute_error, "MSE":mean_squared_error, "R2": r2_score}

## RandomizedSearchCV:

### RandomForestRegressor:

In [34]:
log_table=[]

In [35]:
start = time.time()
reg=RandomForestRegressor()
grid_dict = {'n_estimators': [50, 100, 200],
             "criterion":['squared_error'],
             'max_depth': [5,10,20,100],
             'min_samples_split': [2,4,6,8,10],
             "random_state":[1023,42,3]}

RF_rscv = RandomizedSearchCV(estimator=reg,
                             param_distributions = grid_dict,
                             n_jobs =-1,
                             refit=True,
                             verbose=5)

RF_rscv.fit(X_train_scal,y_train)
y_valid_pred=RF_rscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'RandomizedSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


### DecisionTreeRegressor:

In [36]:
start = time.time()
reg=DecisionTreeRegressor()

grid_dict = {"criterion":['squared_error'],
             "splitter":['best', 'random'],
             "max_depth": [5,10,20,100],
             "min_samples_split": [2,4,6,8,10],
             "random_state":[1023,42,3],
             "max_features" : ['auto','sqrt','log2']}

DT_rscv = RandomizedSearchCV(estimator=reg,
                             param_distributions = grid_dict,
                             n_jobs =-1,
                             refit=True,
                             verbose=5)

DT_rscv.fit(X_train_scal,y_train)
y_valid_pred=DT_rscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'RandomizedSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


### KNeighborsRegressor:

In [37]:
start = time.time()
reg=KNeighborsRegressor()
grid_dict = {"n_neighbors": [1,3,4,5,10,20],
             "weights":['uniform','distance'],
             "algorithm":['auto','ball_tree','kd_tree','brute']}

Knn_rscv = RandomizedSearchCV(estimator=reg,
                              param_distributions = grid_dict,
                              n_jobs =-1,
                              refit=True,
                              verbose=5)

Knn_rscv.fit(X_train_scal,y_train)
y_valid_pred=Knn_rscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'RandomizedSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


# GridSearchCV:

### RandomForestRegressor:

In [38]:
start = time.time()
reg=RandomForestRegressor()
grid_dict = {'n_estimators': [50, 100, 200],
             "criterion":['squared_error'],
             'max_depth': [5,10,20,100],
             'min_samples_split': [2,4,6,8,10],
             "random_state":[1023,42,3]}

RF_gscv = GridSearchCV(estimator=reg,
                       param_grid = grid_dict,
                       n_jobs =-1,
                       refit=True,
                       verbose=5)

RF_gscv.fit(X_train_scal,y_train)
y_pred=RF_gscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'GridSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 180 candidates, totalling 900 fits


### DecisionTreeRegressor:

In [39]:
start = time.time()
reg=DecisionTreeRegressor()

grid_dict = {"criterion":['squared_error'],
             "splitter":['best', 'random'],
             "max_depth": [5,10,20,100],
             "min_samples_split": [2,4,6,8,10],
             "random_state":[1023,42,3],
             "max_features" : ['auto','sqrt','log2']}

DT_gscv = GridSearchCV(estimator=reg,
                       param_grid = grid_dict,
                       n_jobs =-1,
                       refit=True,
                       verbose=5)

DT_gscv.fit(X_train_scal,y_train)
y_pred=DT_gscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'GridSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [40]:
start = time.time()
reg=KNeighborsRegressor()
grid_dict = {"n_neighbors": [1,3,4,5,10,20],
             "weights":['uniform','distance'],
             "algorithm":['auto','ball_tree','kd_tree','brute']}

Knn_gscv = GridSearchCV(estimator=reg,
                        param_grid = grid_dict,
                        n_jobs =-1,
                        refit=True,
                        verbose=5)

Knn_gscv.fit(X_train_scal,y_train)
y_pred=Knn_gscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'GridSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


# BAYESIAN OPTIMIZATION WITH hyperopt package:

### RandomForestRegressor:

In [41]:
space = {"n_estimators": hp.quniform('n_estimators', 50, 200, 1),
         'max_depth': hp.quniform('max_depth', 5, 100, 1),
         'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
         "random_state": hp.choice('random_state',(1023,42,3))}

# Define the objective function
def objective(params):
    reg = RandomForestRegressor( n_estimators=int(params['n_estimators']),
                                 max_depth=int(params['max_depth']),
                                 min_samples_split=int(params['min_samples_split']),
                                 random_state=int(params['random_state']))
    score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': 'ok'}

# Define the search algorithm and search space
trials = Trials()
algo = tpe.suggest

# Run the optimization
start = time.time()
best = fmin(objective, space, algo=algo, trials=trials, max_evals=5,return_argmin=False)

# Train a model with the selected hyperparameters
RF_bopt = RandomForestRegressor( n_estimators=int(best['n_estimators']),
                             max_depth=int(best['max_depth']),
                             min_samples_split=int(best['min_samples_split']),
                             random_state=int(best['random_state']))

RF_bopt.fit(X_train_scal, y_train)
y_valid_pred=RF_bopt.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'BAYESIAN OPTIMIZATION', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

100%|██████████| 5/5 [1:00:53<00:00, 730.67s/trial, best loss: 0.04714660571651517] 


### DecisionTreeRegressor:

In [42]:
space = {'max_depth': hp.quniform('max_depth', 5, 100, 1),
         'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
         "random_state": hp.choice('random_state',(1023,42,3)),
         "splitter" : hp.choice("splitter",['best', 'random']),
         "max_features" : hp.choice("max_features",['auto','sqrt','log2'])}

# Define the objective function
def objective(params):
    reg = DecisionTreeRegressor( max_depth=int(params['max_depth']),
                                 min_samples_split=int(params['min_samples_split']),
                                 random_state=int(params['random_state']),
                                 splitter=str(params['splitter']),
                                 max_features=str(params['max_features']))

    score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': 'ok'}

# Define the search algorithm and search space
trials = Trials()
algo = tpe.suggest

# Run the optimization
start = time.time()
best = fmin(objective, space, algo=algo, trials=trials, max_evals=5,return_argmin=False)

# Train a model with the selected hyperparameters
DT_bopt = DecisionTreeRegressor( max_depth=int(best['max_depth']),
                             min_samples_split=int(best['min_samples_split']),
                             random_state=int(best['random_state']),
                             splitter=str(best['splitter']),
                             max_features=str(best['max_features']))

DT_bopt.fit(X_train_scal, y_train)
y_valid_pred=DT_bopt.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'BAYESIAN OPTIMIZATION', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\an

 60%|██████    | 3/5 [00:09<00:06,  3.08s/trial, best loss: 0.06618976759936544]

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

C:\Users\User\an

100%|██████████| 5/5 [00:16<00:00,  3.22s/trial, best loss: 0.06618976759936544]


C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [43]:
space = {"n_neighbors": hp.quniform('n_neighbors', 1, 20, 1),
         "weights": hp.choice("weights",['uniform','distance']),
         "algorithm":hp.choice("algorithm",['auto','ball_tree','kd_tree','brute'])}

# Define the objective function
def objective(params):
    reg = KNeighborsRegressor(n_neighbors=int(params['n_neighbors']),
                              weights=str(params['weights']),
                              algorithm=str(params['algorithm']),)

    score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': 'ok'}

# Define the search algorithm and search space
trials = Trials()
algo = tpe.suggest

# Run the optimization
start = time.time()
best = fmin(objective, space, algo=algo, trials=trials, max_evals=5,return_argmin=False)

# Train a model with the selected hyperparameters
Knn_bopt = KNeighborsRegressor(n_neighbors=int(best['n_neighbors']),
                                weights=str(best['weights']),
                                algorithm=str(best['algorithm']),)

Knn_bopt.fit(X_train_scal, y_train)
y_valid_pred=Knn_bopt.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'BAYESIAN OPTIMIZATION', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

100%|██████████| 5/5 [00:28<00:00,  5.64s/trial, best loss: 0.048760937477583866]


# GENETIC ALGORITHM OPTIMIZATION:

In [ ]:
# ! pip install sklearn-genetic-opt

### RandomForestRegressor:

In [44]:
reg = RandomForestRegressor()

cv=KFold(n_splits=5, shuffle=True)

search_space = {'n_estimators': Integer(5,200),
                'max_depth':  Integer(5,100),
                'min_samples_split':  Integer(2,10),
                'random_state': Categorical([1023,42,3])}

start = time.time()
RF_ga = GASearchCV(estimator=reg,
                       cv=cv,
                       scoring='r2',
                       param_grid=search_space,
                       n_jobs=-1,
                       verbose=True,
                       population_size=10,
                       generations=3)

RF_ga.fit(X_train_scal, y_train)
y_valid_pred=RF_ga.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'GASearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.848416	0.034671   	0.865235   	0.744893   
1  	20    	0.861386	0.00180862 	0.865235   	0.858548   
2  	20    	0.861445	0.00146575 	0.862839   	0.858188   
3  	20    	0.863096	0.00121355 	0.864678   	0.861258   


### DecisionTreeRegressor:

In [45]:
reg = DecisionTreeRegressor()

cv=KFold(n_splits=5, shuffle=True)

search_space = {'max_depth':  Integer(5,100),
                'min_samples_split':  Integer(2,10),
                'random_state': Categorical([1023,42,3]),
                'splitter' : Categorical(['best', 'random']),
                'max_features' : Categorical(['auto','sqrt','log2'])}

start = time.time()
DT_ga = GASearchCV(estimator=reg,
                       cv=cv,
                       scoring='r2',
                       param_grid=search_space,
                       n_jobs=-1,
                       verbose=True,
                       population_size=10,
                       generations=3)

DT_ga.fit(X_train_scal, y_train)
y_valid_pred=DT_ga.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'GASearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.699923	0.0459025  	0.73482    	0.580949   
1  	20    	0.728627	0.012481   	0.742617   	0.697733   
2  	20    	0.737204	0.00624048 	0.743683   	0.726549   
3  	20    	0.739051	0.00599448 	0.751273   	0.730384   


C:\Users\User\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [46]:
reg = KNeighborsRegressor()

cv=KFold(n_splits=5, shuffle=True)

search_space = {"n_neighbors": Integer(1, 20),
                "weights": Categorical(['uniform','distance']),
                "algorithm":Categorical(['auto','ball_tree','kd_tree','brute'])}

start = time.time()
Knn_ga = GASearchCV(estimator=reg,
                       cv=cv,
                       scoring='r2',
                       param_grid=search_space,
                       n_jobs=-1,
                       verbose=True,
                       population_size=10,
                       generations=3)

Knn_ga.fit(X_train_scal, y_train)
y_valid_pred=Knn_ga.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'GASearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.790619	0.0186854  	0.815405   	0.762359   
1  	20    	0.810723	0.00657157 	0.819897   	0.799174   
2  	20    	0.820176	0.00361397 	0.824093   	0.814828   
3  	20    	0.82141 	0.00518391 	0.825449   	0.806374   


# Results:

In [47]:
log_table = pd.DataFrame(log_table, columns = ['Model','Hyperparameter Algorithm ','Metric','Metric score','Elapsed time'])

In [48]:
log_table[log_table['Metric score']==log_table['Metric score'].max()]

,Model,Hyperparameter Algorithm,Metric,Metric score,Elapsed time
2,RF,RandomizedSearchCV,R2,0.869194,343.810534


### BEST MODEL:

In [49]:
best_model=DT_ga.best_estimator_
best_model

DecisionTreeRegressor(max_depth=41, max_features='auto', min_samples_split=9,
                      random_state=1023)

In [50]:
y_pred=best_model.predict(X_test)

C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


In [51]:
y_pred

array([[0.125     , 0.49896907],
       [0.125     , 0.49896907],
       [0.125     , 0.49896907],
       ...,
       [0.125     , 0.49896907],
       [0.125     , 0.49896907],
       [0.125     , 0.49896907]])

In [52]:
for metric in metric_dict:
    current_metric = metric_dict[metric]
    print([metric, metric_dict[metric](y_test, y_pred)])

['MAE', 0.3374652086170401]
['MSE', 0.15799204713820458]
['R2', -1.4932721477054525]


### Save csv file with results:

In [53]:
log_table.to_csv('log_table1.csv', index=False)

In [ ]:
# from google.colab import files

# files.download('log_table1.csv')